Nella cartella *files* trovi 8 file:
- 2 file di testo
- 2 file audio
- 4 immagini,

con varie estensioni.

**Step 1**

Inizia creando, in un notebook, uno script Python che iteri in ordine alfabetico sui file della cartella files e, a seconda del tipo (audio, documento, immagine), li sposti nella relativa sottocartella (qui sotto trovi un esempio). Se la sottocartella non esiste, il tuo script dovrà crearla automaticamente.

Durante il ciclo, lo script deve stampare le informazioni dei file: nome, tipo e dimensione in byte. Questo è l'output desiderato:

In [7]:
# Librerie in uso nel codice
import os
import shutil
import csv

def get_files_data(files, path, map_type):
    '''
    Ottiene name, size, raw_type, type di ogni file.

    files: lista di nome_file.estensione
    path: percorso (str) dove sono situati i file
    map_type: dizionario mappare tipi file a nome. mp3->audio
    '''

    files_data = {'name':[], 'raw_type':[], 'size':[], 'type':[]}

    for file in files:
        file_path = os.path.join(path, file)

        # Controllo file non è una directory o recap.csv
        if os.path.isfile(file_path) and file!='recap.csv':

            size = os.path.getsize(file_path)
            name, extens = os.path.splitext(file)
            
            if size and name and extens: # Per essere sicuro che i dati esistono
                files_data['name'].append(name)
                files_data['size'].append(size)
                files_data['raw_type'].append(extens)
                good_type = 'None' # Nel caso non c'è l'estensione del file, append 'None'

                for f_type in map_type.keys():
                    if extens in map_type[f_type]:
                        good_type = f_type
                        break
                files_data['type'].append(good_type)

            else:
                print(f'Missing data for {file}!')

    return files_data # Pandas-style


def move_files(files_data, path, map_dir, verbose=True):
    '''
    Sposta i file in una sub-directory in base al type.

    files_data: dizionario stile Pandas. Ogni key-value pair e' una colonna
    path: percorso (str) dove sono situati i file
    map_dir: dizionario mappare file alla cartella. audio->audios
    verbose: stampa diverse info su ogni file se True
    '''

    for idx in range(len(files_data['name'])): # Accedere ad ogni file
        dir_name = None
        for name in map_dir.keys(): # Ottenere nome sub-directory
            if files_data['type'][idx] in map_dir[name]:
                dir_name = name

        dir_path = path + '/' + dir_name
        if not os.path.exists(dir_path): # Verifica esistenza sub-directory
            os.makedirs(dir_path)
        
        # Sposta il file
        curr_path = path + '/' + files_data['name'][idx] + files_data['raw_type'][idx]
        new_path = dir_path + '/' + files_data['name'][idx] + files_data['raw_type'][idx]
        shutil.move(curr_path, new_path)

        if verbose: # stampa i dati del singolo file
            name = files_data['name'][idx]
            file_type = files_data['type'][idx]
            size = files_data['size'][idx]
            print(f'{name} type:{file_type} size:{size}B')


def get_recap(files_data, path):
    '''
    Crea un file csv con le info dei files.

    files_data: dizionario stile Pandas. Ogni key-value pair e' una colonna
    path: percorso (str) dove si vuole salvare il file csv
    name: nome del file csv senza estensione
    '''

    file_path = os.path.join(path, "recap.csv")

    # Verifica se il file esiste già
    mode = 'a' if os.path.exists(file_path) else 'w'

    # Se recap non esiste, lo crea. Altrimenti aggiunge le info
    with open(file_path, mode, newline='') as csvfile:
        fieldnames = ['name', 'type', 'size (B)']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        if mode == 'w':
            writer.writeheader()

        # Scriviamo i dati dei file nel CSV
        for i in range(len(files_data['name'])):
            writer.writerow({
                'name': files_data['name'][i],
                'type': files_data['type'][i],
                'size (B)': files_data['size'][i]
            })


if __name__ == '__main__':
    # Ottiene i file in ordine alfabetico
    path = './files'
    files = sorted(os.listdir(path))

    map_type = {
        'audio': {'.mp3', '.wav', '.flac'},
        'doc': {'.txt', '.odt', '.doc', '.docx', '.pdf'},
        'image': {'.jpg', '.jpeg', '.png', '.gif', '.bmp', '.svg'},
        'video': {'.mp4', '.avi', '.mov', '.mkv', '.wmv'}
    }

    map_dir = {
        'audios': {'audio'},
        'docs': {'doc'},
        'images': {'image'},
        'videos': {'video'}
    }

    files_data = get_files_data(files, path, map_type)
    move_files(files_data, path, map_dir)
    get_recap(files_data, path)

['audios', 'docs', 'images', 'recap.csv', 'song1.mp3', 'song2.mp3', 'trump.jpeg']
song1 type:audio size:1087849B
song2 type:audio size:764176B
trump type:image size:10195B


Oltre a stamparne le informazioni via via che li sposti, tieni traccia dei file creando un documento *recap.csv* con le stesse informazioni. Trovi un esempio in questa cartella.

La struttura finale della cartella files dovrà essere:

        - files            
            - audio
                - song1.mp3
                - song2.mp3
            - docs
                - ciao.txt
                - pippo.odt
            - images
                - bw.png
                - daffodil.jpg
                - eclipse.png
                - trump.jpeg    
            - recap.csv

Commenta il codice con i passaggi che fai. Questo vale anche per i prossimi Step.

**Attenzione**: lo script, ogni volta che viene lanciato per spostare nuovi file, deve *aggiornare* (e non sovrascrivere) le sottocartelle e il file di recap. Per controllare che tutto funzioni correttamente, puoi aggiungere altri file alla cartella files e fare un test; oppure, puoi dividere gli 8 file originali in due gruppi e lasciarne uno per il test.

**Consiglio**: puoi usare le librerie *os*, *shutil* e *csv*. 
                
---

**Step 2**

Inserisci lo script che hai creato in un piccolo eseguibile (chiamalo *addfile.py* e posizionalo in questa cartella, a fianco del notebook) dotato di *interfaccia a linea di comando* (CLI).

Lo scopo dell'eseguibile è spostare un *singolo* file (che si trova nella cartella files) nella sottocartella di competenza, aggiornando il recap.

L'interfaccia dell'eseguibile ha come unico argomento (obbligatorio) il nome del file da spostare (comprensivo di formato, es: 'trump.jpeg'). Nel caso in cui il file passato come argomento non esista, l'interfaccia deve comunicarlo all'utente.

**Consiglio**: oltre alle precedenti, puoi usare le librerie *sys* e *argparse*.

---

**Step 3**

Una immagine in scala di grigio ha un solo livello di colore, una RGB ne ha 3, una RGBA 4 (l'ultimo è detto canale *alpha*).

Il modulo *Image* della libreria *PIL* permette di caricare un'immagine, che può essere trasformata in un array NumPy attraverso la funzione *np.array*. A partire da tale array, è possibile capire se l'immagine caricata è in scala di grigio, RGB o RGBA.

Aggiungi al notebook dello Step 1 uno script che iteri sulla sottocartella *images* e costruisca una tabella riassuntiva come questa (prodotta con la libreria *tabulate*):

In [11]:
#FILL ME

╒══════════╤══════════╤═════════╤═════════════╤════════╤════════╤═══════╤═════════╕
│ name     │   height │   width │   grayscale │      R │      G │     B │   ALPHA │
╞══════════╪══════════╪═════════╪═════════════╪════════╪════════╪═══════╪═════════╡
│ bw       │      512 │     512 │       21.48 │   0.00 │   0.00 │  0.00 │    0.00 │
├──────────┼──────────┼─────────┼─────────────┼────────┼────────┼───────┼─────────┤
│ daffodil │      500 │     335 │        0.00 │ 109.25 │  85.56 │  4.97 │    0.00 │
├──────────┼──────────┼─────────┼─────────────┼────────┼────────┼───────┼─────────┤
│ eclipse  │      256 │     256 │        0.00 │ 109.05 │ 109.52 │ 39.85 │  133.59 │
├──────────┼──────────┼─────────┼─────────────┼────────┼────────┼───────┼─────────┤
│ trump    │      183 │     275 │        0.00 │  97.01 │  98.99 │ 90.92 │    0.00 │
╘══════════╧══════════╧═════════╧═════════════╧════════╧════════╧═══════╧═════════╛


Oltre al nome del file, la tabella riporta:

- altezza dell'immagine, in pixel
- larghezza dell'immagine, in pixel
- se l'immagine è in scala di grigio, la colonna *grayscale* indica la media dei valori dell'unico livello di colore
- se l'immagine è a colori, le altre colonne indicano la media dei valori di ogni livello di colore.

---

**Dovrai consegnare**:
- un notebook con gli Step 1 e 3; per semplicità puoi chiamarlo come questo
- addfile.py con quanto richiesto dallo Step 2.